# Fraud PoC — DuckDB + Embeddings + Ollama

This notebook demonstrates an end-to-end PoC:
- create canonical tables in DuckDB
- ingest synthetic transactions
- compute embeddings (sentence-transformers)
- build a local ANN index (faiss / hnswlib / sklearn fallback)
- perform retrieval, assemble prompt and call Ollama (local LLM)
- persist LLM result and provenance back into DuckDB

Notes:
- Ensure your environment has the dependencies from `requirements.txt` installed.
- Ensure Ollama is running and reachable at OLLAMA_URL (default http://localhost:11434/api/generate).
- The notebook auto-falls back to hnswlib or sklearn NearestNeighbors if faiss is unavailable.

In [1]:
# Configuration & imports
import os
import json
import uuid
import time
import hashlib
from typing import List, Tuple, Dict, Any

import duckdb
import pandas as pd
import numpy as np
import requests

# Embedding lib
from sentence_transformers import SentenceTransformer

# ANN fallbacks
HAS_FAISS = False
try:
    import faiss
    HAS_FAISS = True
except Exception:
    HAS_FAISS = False

HAS_HNSW = False
try:
    import hnswlib
    HAS_HNSW = True
except Exception:
    HAS_HNSW = False

from sklearn.neighbors import NearestNeighbors

# Ollama endpoint (adjust if different in your setup)
OLLAMA_URL = os.environ.get('OLLAMA_URL', 'http://localhost:11434/api/generate')
OLLAMA_MODEL = os.environ.get('OLLAMA_MODEL', 'olmo-3')  # replace with your pulled model name

# DuckDB path
DB_PATH = os.environ.get('DB_PATH', 'fraud_poc.duckdb')
print(DB_PATH)


fraud_poc.duckdb


In [2]:
# Utilities: prompt hashing and calling Ollama
def prompt_hash(prompt_text: str) -> str:
    return hashlib.sha256(prompt_text.encode('utf-8')).hexdigest()

def call_ollama(model: str, prompt_text: str, temperature: float = 0.0, timeout: int = 60) -> Dict[str, Any]:
    payload = {
        'model': model,
        'prompt': prompt_text,
        'temperature': temperature
    }
    start = time.time()
    resp = requests.post(OLLAMA_URL, json=payload, timeout=timeout)
    resp.raise_for_status()
    elapsed_ms = int((time.time() - start) * 1000)
    try:
        data = resp.json()
    except ValueError:
        data = {'text': resp.text}
    # Normalize text extraction (common keys)
    text = data.get('text') or data.get('response') or data.get('content') or resp.text
    return {
        'llm_provider': 'ollama_local',
        'llm_model': model,
        'llm_response_raw': text,
        'llm_response_json': data if isinstance(data, dict) else None,
        'latency_ms': elapsed_ms,
        'prompt_hash': prompt_hash(prompt_text),
        'call_id': str(uuid.uuid4())
    }

def try_parse_json(s: str):
    try:
        return json.loads(s)
    except Exception:
        return None

In [3]:
# Create DuckDB connection and required tables (idempotent)
con = duckdb.connect(DB_PATH)

CREATE_TRANSACTIONS = '''
CREATE TABLE IF NOT EXISTS transactions (
  tx_id VARCHAR PRIMARY KEY,
  account_id VARCHAR,
  amount DOUBLE,
  currency VARCHAR,
  merchant VARCHAR,
  description VARCHAR,
  timestamp TIMESTAMP,
  ingestion_job_id VARCHAR,
  raw_source VARCHAR,
  pii_masked BOOLEAN DEFAULT FALSE,
  created_at TIMESTAMP DEFAULT current_timestamp
);
'''

CREATE_EMBEDDINGS = '''
CREATE TABLE IF NOT EXISTS embeddings (
  tx_id VARCHAR PRIMARY KEY,
  emb_json TEXT,
  emb_model VARCHAR,
  emb_created_at TIMESTAMP DEFAULT current_timestamp,
  emb_job_id VARCHAR
);
'''

CREATE_LLM = '''
CREATE TABLE IF NOT EXISTS llm_results (
  id VARCHAR PRIMARY KEY,
  tx_id VARCHAR,
  llm_model VARCHAR,
  llm_provider VARCHAR,
  llm_prompt_hash VARCHAR,
  llm_prompt VARCHAR,
  llm_response VARCHAR,
  parsed_response JSON,
  risk_score DOUBLE,
  evidence_tx_ids JSON,
  call_latency_ms INTEGER,
  provenance JSON,
  created_at TIMESTAMP DEFAULT current_timestamp
);
'''

con.execute(CREATE_TRANSACTIONS)
con.execute(CREATE_EMBEDDINGS)
con.execute(CREATE_LLM)
print('Created tables (if not existing) in', DB_PATH)

Created tables (if not existing) in fraud_poc.duckdb


In [4]:
# Create synthetic transactions and insert into DuckDB
import datetime

n = 200
rng = np.random.default_rng(42)
txs = []
for i in range(n):
    tx_id = f"tx_{i+1:06d}"
    account = f"acct_{rng.integers(1,50):04d}"
    amount = float(np.round(rng.normal(50, 120), 2))
    merchant = rng.choice(['store_a','store_b','online_shop','gas_station','restaurant'])
    desc = f"purchase at {merchant} for ${amount:.2f}"
    ts = datetime.datetime.utcnow() - datetime.timedelta(minutes=int(rng.integers(0, 60*24)))
    txs.append((tx_id, account, amount, 'USD', merchant, desc, ts, 'ingest_0', 'synthetic', False))

df = pd.DataFrame(txs, columns=['tx_id','account_id','amount','currency','merchant','description','timestamp','ingestion_job_id','raw_source','pii_masked'])

con.register('df_tx', df)
con.execute("INSERT OR REPLACE INTO transactions (tx_id, account_id, amount, currency, merchant, description, timestamp, ingestion_job_id, raw_source, pii_masked) SELECT * FROM df_tx")
print('Inserted', len(df), 'transactions')

Inserted 200 transactions


/tmp/ipython-input-910719176.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow() - datetime.timedelta(minutes=int(rng.integers(0, 60*24)))


In [5]:
# Compute embeddings with sentence-transformers and store in embeddings table
embed_model_name = 'all-MiniLM-L6-v2'
print('Loading embedding model:', embed_model_name)
embedder = SentenceTransformer(embed_model_name)

# Load transactions into a DataFrame (from DuckDB)
df_tx = con.execute('SELECT tx_id, description FROM transactions').df()
texts = df_tx['description'].tolist()
vectors = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True)

import json
rows = []
for tx_id, vec in zip(df_tx['tx_id'].tolist(), vectors):
    rows.append((tx_id, json.dumps(vec.tolist()), embed_model_name, None))

df_emb = pd.DataFrame(rows, columns=['tx_id','emb_json','emb_model','emb_job_id'])
con.register('df_emb', df_emb)
con.execute("INSERT OR REPLACE INTO embeddings (tx_id, emb_json, emb_model, emb_job_id) SELECT * FROM df_emb")
print('Stored', len(df_emb), 'embeddings in DuckDB')

Loading embedding model: all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Stored 200 embeddings in DuckDB


In [6]:
# Build an in-memory ANN index (faiss preferred, then hnswlib, then sklearn)
emb_matrix = np.vstack([np.array(json.loads(x)).astype('float32') for x in con.execute('SELECT emb_json FROM embeddings ORDER BY emb_created_at').df()['emb_json']])
ids = con.execute('SELECT tx_id FROM embeddings ORDER BY emb_created_at').df()['tx_id'].tolist()
id_to_index = {tx_id: idx for idx, tx_id in enumerate(ids)}

ann_index = None
ann_backend = None
dim = emb_matrix.shape[1]

if HAS_FAISS:
    import faiss
    # normalize for cosine similarity
    xb = emb_matrix.copy()
    faiss.normalize_L2(xb)
    index = faiss.IndexFlatIP(dim)
    index.add(xb)
    ann_index = index
    ann_backend = 'faiss'
    print('Built FAISS IndexFlatIP with', xb.shape[0], 'vectors')
elif HAS_HNSW:
    import hnswlib
    p = hnswlib.Index(space='cosine', dim=dim)
    p.init_index(max_elements=emb_matrix.shape[0], ef_construction=200, M=16)
    p.add_items(emb_matrix, np.arange(emb_matrix.shape[0]))
    p.set_ef(50)
    ann_index = p
    ann_backend = 'hnswlib'
    print('Built hnswlib index with', emb_matrix.shape[0], 'vectors')
else:
    # sklearn brute-force as fallback
    nbrs = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute').fit(emb_matrix)
    ann_index = nbrs
    ann_backend = 'sklearn'
    print('Built sklearn NearestNeighbors fallback (brute)')


Built sklearn NearestNeighbors fallback (brute)


In [7]:
# Retrieval helper
def retrieve_topk(query_vec: np.ndarray, k: int = 5) -> List[Tuple[str, float]]:
    q = query_vec.astype('float32')
    if ann_backend == 'faiss':
        qn = q.copy()
        faiss.normalize_L2(qn)
        D, I = ann_index.search(qn.reshape(1, -1), k)
        # FAISS IndexFlatIP returns inner product; since we normalized, it's cosine similarity
        return [(ids[int(i)], float(D[0][idx])) for idx, i in enumerate(I[0])]
    elif ann_backend == 'hnswlib':
        labels, distances = ann_index.knn_query(q, k=k)
        return [(ids[int(lbl)], float(dist)) for lbl, dist in zip(labels[0], distances[0])]
    else:
        D, I = ann_index.kneighbors(q.reshape(1, -1), n_neighbors=k, return_distance=True)
        return [(ids[int(i)], float(D[0][idx])) for idx, i in enumerate(I[0])]


In [8]:
def retrieve_topk_safe(q_vec, k=5, con=None, embeddings_table="embeddings"):
    """
    Defensive top-k retrieval.
    - q_vec: 1D array-like vector
    - k: number results
    - con: duckdb connection or None; if None we open a temporary connection (adjust as needed)
    - embeddings_table: table name; assumed columns (tx_id, emb_json) where emb_json is JSON array
    Returns list of tuples (tx_id, score, metadata_if_any)
    """
    # normalize q_vec to 1D numpy array
    q = np.asarray(q_vec)
    if q.ndim > 1 and q.shape[0] == 1:
        q = q.reshape(-1)
    if q.ndim != 1:
        raise ValueError(f"Expected 1D q_vec, got shape {q.shape}")

    # get rows from DB - FIX: use tx_id instead of id, and emb_json instead of embedding
    if con is None:
        con = duckdb.connect()  # adjust to your DB path or pass your existing connection

    try:
        # Match your actual schema: tx_id, emb_json, emb_model
        rows = con.execute(f"SELECT tx_id, emb_json FROM {embeddings_table}").fetchall()
    except Exception as e:
        print(f"Error fetching embeddings: {e}")
        return []

    # defensive processing
    scored = []
    for i, row in enumerate(rows):
        if row is None or not isinstance(row, (tuple, list)):
            continue
        # ensure we have at least tx_id and emb_json
        if len(row) < 2:
            continue

        tx_id = row[0]
        emb_json = row[1]

        # Parse JSON string to numpy array
        try:
            vec = np.asarray(json.loads(emb_json), dtype=float)
        except Exception:
            # couldn't parse embedding JSON; skip
            continue

        # guard shapes
        if vec.shape != q.shape:
            # attempt reshape if needed
            try:
                vec = vec.reshape(q.shape)
            except Exception:
                # incompatible shape
                continue

        # compute cosine similarity (normalized dot product)
        q_norm = np.linalg.norm(q)
        vec_norm = np.linalg.norm(vec)
        if q_norm > 0 and vec_norm > 0:
            score = float(np.dot(q, vec) / (q_norm * vec_norm))
        else:
            score = 0.0

        scored.append((tx_id, score))

    # sort and return top-k
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:k]

In [18]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tgz
######################################################################## 100.0%
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [20]:
!ollama pull gemma3
!ollama serve


Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [29]:
# Simulate an incoming transaction, retrieve similar historical cases, call Ollama, and persist result
incoming = {
    'tx_id': 'tx_live_0001',
    'account_id': 'acct_9999',
    'amount': 399.99,
    'currency': 'USD',
    'merchant': 'online_shop',
    'description': 'large purchase at online_shop',
}
# compute embedding for incoming description
q_vec = embedder.encode([incoming['description']], convert_to_numpy=True)[0].astype('float32')
print(q_vec)
topk = retrieve_topk_safe(q_vec, k=5, con=con, embeddings_table="embeddings")
print("topk:", topk)

# assemble a RAG prompt (simple template)
retrieved_texts = []
for txid, score in topk:
    row = con.execute("SELECT tx_id, account_id, amount, merchant, description FROM transactions WHERE tx_id = ?", (txid,)).fetchdf()
    if len(row) > 0:
        r = row.iloc[0]
        retrieved_texts.append(f"- {r.tx_id} | {r.merchant} | ${r.amount:.2f} | {r.description}")

prompt = f"You are a fraud analyst assistant. Given the incoming transaction and similar historical transactions, output JSON with fields: {json.dumps({'risk_score':'float 0..1','explanation':'string','evidence_tx_ids':'list'})}.\n\n"
prompt += "Incoming transaction:\n"
prompt += f"{incoming['tx_id']} | {incoming['merchant']} | ${incoming['amount']:.2f} | {incoming['description']}\n\n"
prompt += "Retrieved similar historical transactions:\n"
prompt += "\n".join(retrieved_texts)
prompt += "\n\nReturn only valid JSON."

print('\nPrompt preview:\n', prompt[:1000])

# Call Ollama with retry logic
max_retries = 3
retry_delay = 5
timeout = 180  # Increase timeout to 3 minutes

for attempt in range(max_retries):
    try:
        resp = call_ollama(OLLAMA_MODEL, prompt, temperature=0.0, timeout=timeout)
        print('Ollama call latency (ms):', resp['latency_ms'])
        break
    except requests.exceptions.ReadTimeout:
        if attempt < max_retries - 1:
            wait_time = retry_delay * (2 ** attempt)
            print(f'Timeout on attempt {attempt + 1}/{max_retries}. Retrying in {wait_time}s...')
            time.sleep(wait_time)
        else:
            print(f'Failed after {max_retries} attempts. Generating fallback response.')
            resp = {
                'llm_provider': 'ollama_local',
                'llm_model': OLLAMA_MODEL,
                'llm_response_raw': '{"risk_score": 0.5, "explanation": "Timeout - unable to analyze", "evidence_tx_ids": []}',
                'llm_response_json': None,
                'latency_ms': timeout * 1000,
                'prompt_hash': prompt_hash(prompt),
                'call_id': str(uuid.uuid4())
            }
    except Exception as e:
        print(f'Error calling Ollama: {e}')
        if attempt < max_retries - 1:
            wait_time = retry_delay * (2 ** attempt)
            print(f'Retrying in {wait_time}s...')
            time.sleep(wait_time)
        else:
            raise

# Try to parse JSON from response
parsed = try_parse_json(resp['llm_response_raw'])
risk_score = None
evidence = None
if parsed and isinstance(parsed, dict):
    risk_score = parsed.get('risk_score')
    evidence = parsed.get('evidence_tx_ids')
else:
    # if response not JSON, keep raw text
    parsed = {'text': resp['llm_response_raw']}

# Persist into llm_results
row_id = resp['call_id']

# Extract transaction IDs from topk results
retrieved_ids = [txid for txid, score in topk]

provenance = {
    'emb_model': embed_model_name,
    'index_backend': ann_backend,
    'retrieved': retrieved_ids
}
con.execute(
    """
    INSERT INTO llm_results (id, tx_id, llm_model, llm_provider, llm_prompt_hash, llm_prompt, llm_response, parsed_response, risk_score, evidence_tx_ids, call_latency_ms, provenance)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    (
        row_id,
        incoming['tx_id'],
        resp['llm_model'],
        resp['llm_provider'],
        resp['prompt_hash'],
        prompt if os.environ.get('STORE_PROMPTS','1')=='1' else None,
        resp['llm_response_raw'],
        json.dumps(parsed),
        float(risk_score) if risk_score is not None else None,
        json.dumps(evidence) if evidence is not None else json.dumps(retrieved_ids),
        int(resp['latency_ms']),
        json.dumps(provenance)
    )
)
print('Inserted llm_results id=', row_id)

[-2.33178679e-02  4.36213166e-02 -2.96767596e-02 -2.38433331e-02
  3.16798873e-02 -3.29891741e-02  1.99094005e-02  8.80701169e-02
 -4.87000123e-02  1.89901050e-02  6.81477189e-02  7.00053200e-02
 -1.75665244e-02 -7.66963884e-02  2.75933146e-02 -1.58611219e-02
  3.55156809e-02 -5.25272265e-02 -2.33377591e-02 -4.94899414e-02
 -5.46921380e-02 -2.23579984e-02  1.63522772e-02  5.45342490e-02
 -2.60658823e-02 -2.36302968e-02 -2.80885324e-02  3.04723959e-02
  2.05745059e-03 -1.01760752e-01 -9.36757307e-03  6.86745122e-02
  8.00208151e-02  2.78094448e-02 -1.06129134e-02 -2.18146518e-02
 -5.74116446e-02 -8.46812651e-02 -6.19768770e-03 -3.69383730e-02
  5.57370000e-02  1.63095128e-02 -5.85012361e-02  3.43981646e-02
  2.47597098e-02  3.76512446e-02  1.03834830e-02  1.06246598e-01
  5.54109216e-02  6.80532828e-02 -6.85372874e-02  3.78160253e-02
 -4.22982164e-02  1.42521085e-03 -1.03825115e-01 -4.58065160e-02
  2.23871153e-02 -5.65554872e-02 -7.50771724e-04 -2.49073058e-02
  4.00769413e-02 -3.24188

In [22]:
import subprocess
import os

# Find processes using port 11434 and kill them
try:
    # Use lsof to find PIDs listening on port 11434
    # -t: output process IDs only
    # -i: select by Internet address (port)
    # -s TCP:LISTEN: select only TCP processes in LISTEN state
    pids_output = subprocess.check_output(['lsof', '-t', '-i', ':11434', '-s', 'TCP:LISTEN'], stderr=subprocess.STDOUT)
    pids = pids_output.decode().strip().split('\n')
    pids = [p for p in pids if p] # Filter out empty strings

    if pids:
        print(f"Found processes using port 11434: {', '.join(pids)}")
        for pid in pids:
            try:
                os.kill(int(pid), 9)  # Send SIGKILL signal
                print(f"Killed process {pid}")
            except OSError as e:
                print(f"Error killing process {pid}: {e}")
    else:
        print("No processes found using port 11434.")
except subprocess.CalledProcessError as e:
    print(f"Could not find processes using port 11434: {e.output.decode().strip()}")
except FileNotFoundError:
    print("lsof command not found. Cannot automatically kill processes.")
    print("Please manually ensure no other process is using port 11434 before restarting Ollama.")

Found processes using port 11434: 19839
Killed process 19839


To run Ollama in the background, you can use `nohup` with the `&` operator. This will keep Ollama running even if the cell execution finishes or you close the notebook.

In [28]:
# Pull the 'olmo-3' model
!ollama pull olmo-3

In [30]:
# Kill any existing Ollama processes before starting a new one in the background
!lsof -t -i :11434 -s TCP:LISTEN | xargs -r kill -9

# Run ollama serve in the background using nohup
!nohup ollama serve > ollama.log 2>&1 &

print("Ollama server started in the background. Check ollama.log for output.")
print("You can also check running processes with `!ps aux | grep ollama`")

Ollama server started in the background. Check ollama.log for output.
You can also check running processes with `!ps aux | grep ollama`


In [31]:
##con.execute('DETACH fraud_poc.duckdb');
# Inspect persisted LLM results
df_llm = con.execute('SELECT id, tx_id, llm_model, llm_provider, risk_score, created_at FROM llm_results ORDER BY created_at DESC LIMIT 10').df()
df_llm

,id,tx_id,llm_model,llm_provider,risk_score,created_at
0,e58ba27c-2397-4f88-8706-42c7e5dff96b,tx_live_0001,olmo-3,ollama_local,NaN,2026-01-10 19:17:00.736


Next steps / notes:
- Tune prompt templates and JSON schema enforcement for robust parsing.
- Add retries, timeouts, and fallback for Ollama calls in production tasks.
- Move index persistence to a vector DB (Milvus/Weaviate) when scaling beyond a single node.
- Mask PII consistently and store PII mappings in a secure vault (HashiCorp Vault) if required.
